In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import random
import datetime as dt

from copy import deepcopy as copy
from client_redis import Client


In [3]:
s = Client(channels=['channel-1'], debug=False)

In [4]:
class Output:
    """
    Helper class
    """
    def __init__(self, name='my-display', max_lines=10):
        self.h = display(display_id=name)
        self.content = ''
        self.mime_type = None
        self.max_lines = max_lines
        self.dic_kind = {'text': 'text/plain'}
    def display(self):
        self.h.display({'text/plain': ''}, raw=True)
    def limit(self, text):
        lines = text.split('\n')
        lines = lines[-self.max_lines:]
        return '\n'.join(lines)
    def _build_obj(self, content, kind, append, new_line):
        self.mime_type = self.dic_kind.get(kind)
        if not self.mime_type:
            return content, False
        if append:
            sep = '\n' if new_line else ''
            self.content = self.limit(self.content + sep + content)
        else:
            self.content = content
        return {self.mime_type: self.content}, True  
    def update(self, content, kind=None, append=False, new_line=True):
        obj, raw = self._build_obj(content, kind, append, new_line)
        self.h.update(obj, raw=raw)
        
out = Output(name='subscribe-status')
show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)

received on channel-1: vector2 --> {"x":105,"y":-26} (14:54:22.62)
received on channel-1: vector2 --> {"x":112,"y":-30} (14:54:22.63)
received on channel-1: vector2 --> {"x":116,"y":-33} (14:54:22.65)
received on channel-1: vector2 --> {"x":126,"y":-38} (14:54:22.67)
received on channel-1: vector2 --> {"x":131,"y":-43} (14:54:22.68)
received on channel-1: vector2 --> {"x":137,"y":-48} (14:54:22.70)
received on channel-1: vector2 --> {"x":140,"y":-49} (14:54:22.72)
received on channel-1: vector2 --> {"x":143,"y":-51} (14:54:22.73)
received on channel-1: vector2 --> {"x":144,"y":-51} (14:54:22.75)
received on channel-1: vector2 --> {"x":147,"y":-51} (14:54:22.77)

In [5]:
out.display()

received on channel-1: vector2 --> {"x":105,"y":-26} (14:54:22.62)
received on channel-1: vector2 --> {"x":112,"y":-30} (14:54:22.63)
received on channel-1: vector2 --> {"x":116,"y":-33} (14:54:22.65)
received on channel-1: vector2 --> {"x":126,"y":-38} (14:54:22.67)
received on channel-1: vector2 --> {"x":131,"y":-43} (14:54:22.68)
received on channel-1: vector2 --> {"x":137,"y":-48} (14:54:22.70)
received on channel-1: vector2 --> {"x":140,"y":-49} (14:54:22.72)
received on channel-1: vector2 --> {"x":143,"y":-51} (14:54:22.73)
received on channel-1: vector2 --> {"x":144,"y":-51} (14:54:22.75)
received on channel-1: vector2 --> {"x":147,"y":-51} (14:54:22.77)

In [6]:
s.subscribe(action=show, test=lambda name: name=='vector2')

In [10]:
s.store['channel-1']['vector2']

'{"x":164,"y":-56}'

In [12]:
s.subscribe_stop()